# Data Processing
This notebook is used to process the data from the raw datasets to curated final datasets and then loaded into a Knowledge Graph.

## 1. Setup
### 1.1. Importing Libraries
The first step is to import the necessary libraries for data processing and define the paths to the raw datasets and intermediate saved datasets.

In [8]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re

datadir = "data/original"
savedir = "data/processed"

### 1.2. Loading Data
The second step is to load the raw datasets into the notebook. Each of them is accessed and stored in a pandas DataFrame.

In [9]:
# Load all data
population_data = pd.read_csv(os.path.join(datadir, "ABS_Population_and_people_by_2021_SA2_(Beta).csv"))

data_distance_all = pd.read_pickle(os.path.join(datadir, "duration_sa2_hospitals.pkl"))
data_distance_shortest = pd.read_pickle(os.path.join(datadir, "duration_sa2_hospital_shortest.pkl"))

hospital_details = pd.read_csv(os.path.join(datadir, "myhospitals-contact-details.csv"), encoding="latin1")


## 2 Preprocessing datasets
### 2.1 Data cleaning

#### 2.1.1 Accessibility

In [21]:
# Create Accessibility dataframe
col1 = []
col2 = []
col3 = []
col4 = []

# Loop through all SA2 distances
for row in tqdm(range(len(data_distance_all)-1)):
    row += 1
    
    # Get the sa2 code
    sa2 = [int(data_distance_all.transpose().iloc[:,row].values[0])]*1011
    col1 = col1 + sa2
    
    # Get the hospital codes
    hospitals = list(range(1,1012,1))
    col2 = col2 + hospitals
    
    # Get the accessibility by checking if the distance is less than 30 minutes
    accessibility = list(data_distance_all.transpose().iloc[1:,1].values < 1800)
    col3 = col3 + accessibility
    
    # Get if the distance is further than 2 hours
    too_far = list(data_distance_all.transpose().iloc[1:,1].values > 7200)
    col4 = col4 + too_far

# Create the dataframe
accessibility_df = pd.DataFrame({'SA2_5DIG': col1, 'hospital_ID':col2, 'accessible':col3, 'further_than_2h':col4})
accessibility_df.head()

100%|██████████| 2309/2309 [01:49<00:00, 21.07it/s]


,SA2_5DIG,hospital_ID,accessible,further_than_2h
0,11008,1,False,True
1,11008,2,False,True
2,11008,3,False,True
3,11008,4,False,True
4,11008,5,False,True


In [22]:
# Save data
file_path = os.path.join(savedir, "AccessibilityEdges.csv")
accessibility_df.to_csv(file_path, index=False)

## 3 Merge datasets
### 3.1 Population Distribution and State Area level 2 (SA2) metadata
The SA2 metadata is created by calculating the population distribution for each SA2 area. The population distribution is calculated by dividing the population of each SA2 area by the total population of the state. The age data is binned into 5-year age groups. The metadata is then merged with the SA2 dataset.

In [23]:
# Take a look at the data
population_data.head()

,OBJECTID,SA2_CODE_2021,SA2_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021,ERP_P_202021,ERP_212021,ERP_M_202021,ERP_F_202021,ERP_192021,...,CENSUS_302021,CENSUS_312021,CENSUS_392021,CENSUS_332021,ADFS_22021,ADFS_32021,ADFS_42021,ADFS_52021,SHAPE_Length,SHAPE_Area
0,1,101021007,Braidwood,3418.3525,https://linked.data.gov.au/dataset/asgsed3/SA2...,4330.0,1.3,2248.0,2082.0,49.9,...,3.5,7.8,180.0,4.1,20.0,182.0,0.5,5.0,3.913695,0.339397
1,2,101021008,Karabar,6.9825,https://linked.data.gov.au/dataset/asgsed3/SA2...,8546.0,1223.9,4324.0,4222.0,36.3,...,5.2,5.7,1228.0,14.4,109.0,257.0,1.6,3.7,0.138642,0.000693
2,3,101021009,Queanbeyan,4.7620,https://linked.data.gov.au/dataset/asgsed3/SA2...,11370.0,2387.7,5788.0,5582.0,36.3,...,14.2,4.9,2615.0,23.1,150.0,354.0,1.6,3.7,0.107150,0.000472
3,4,101021010,Queanbeyan - East,13.0032,https://linked.data.gov.au/dataset/asgsed3/SA2...,5093.0,391.7,2671.0,2422.0,36.2,...,11.9,4.5,1037.0,20.4,152.0,197.0,3.5,4.6,0.189549,0.001290
4,5,101021012,Queanbeyan West - Jerrabomberra,13.6748,https://linked.data.gov.au/dataset/asgsed3/SA2...,12743.0,931.9,6387.0,6356.0,36.6,...,4.1,3.0,1597.0,12.5,585.0,587.0,5.8,5.9,0.193368,0.001356


In [24]:
# ---------------------------------- compute percentages ----------------------------------

# compute percentages ASSUMING THAT THE NAN VALUES ARE EQUAL TO 0, BECAUSE 2021 POPULATION IS EQUAL TO 25.69mil WHICH IS ALMOST EQUAL TO 25.688.079
tot_people21 = population_data["ERP_P_202021"].sum()
population_data["ERP_P_202021 (%)"] = [(population_data.loc[i,"ERP_P_202021"]/tot_people21)*100 if not pd.isna(population_data.loc[i,"ERP_P_202021"]) else np.nan for i in range(len(population_data))]

# compute percentages for male population
totM_people21 = population_data["ERP_M_202021"].sum()
population_data["ERP_M_202021 (%)"] = [(population_data.loc[i,"ERP_M_202021"]/totM_people21)*100 if not pd.isna(population_data.loc[i,"ERP_M_202021"]) else np.nan for i in range(len(population_data))]

# compute percentages for female population
totF_people21 = population_data["ERP_F_202021"].sum()
population_data["ERP_F_202021 (%)"] = [(population_data.loc[i,"ERP_F_202021"]/totF_people21)*100 if not pd.isna(population_data.loc[i,"ERP_F_202021"]) else np.nan for i in range(len(population_data))]
population_data.head()

,OBJECTID,SA2_CODE_2021,SA2_5DIG21,SA2_CODE_2016,SA2_5DIG16,SA2_NAME_2021,AREA_ALBERS_SQKM,ASGS_LOCI_URI_2021,ERP_P_202021,ERP_P_202021 (%),...,ADFS_42021,ADFS_52021,SHAPE_Length,SHAPE_Area,ERP_P_202021 (%),ERP_M_202021 (%),ERP_F_202021 (%),SA2_CODE_2016,SA2_5DIG16,SA2_5DIG21
0,1,101021007,11007,101021007,11007,Braidwood,3418.3525,https://linked.data.gov.au/dataset/asgsed3/SA2...,4330.0,0.016856,...,0.5,5.0,3.913695,0.339397,0.016856,0.017632,0.016092,101021007,11007,11007
1,2,101021008,11008,101021008,11008,Karabar,6.9825,https://linked.data.gov.au/dataset/asgsed3/SA2...,8546.0,0.033268,...,1.6,3.7,0.138642,0.000693,0.033268,0.033915,0.032631,101021008,11008,11008
2,3,101021009,11009,101021009,11009,Queanbeyan,4.7620,https://linked.data.gov.au/dataset/asgsed3/SA2...,11370.0,0.044262,...,1.6,3.7,0.107150,0.000472,0.044262,0.045398,0.043143,101021009,11009,11009
3,4,101021010,11010,101021010,11010,Queanbeyan - East,13.0032,https://linked.data.gov.au/dataset/asgsed3/SA2...,5093.0,0.019826,...,3.5,4.6,0.189549,0.001290,0.019826,0.020950,0.018719,101021010,11010,11010
4,5,101021012,11012,101021012,11012,Queanbeyan West - Jerrabomberra,13.6748,https://linked.data.gov.au/dataset/asgsed3/SA2...,12743.0,0.049607,...,5.8,5.9,0.193368,0.001356,0.049607,0.050096,0.049125,101021012,11012,11012


In [25]:
# Create columns for filtering
columns = [
    'SA2_5DIG16',
    'SA2_NAME_2021',
    'AREA_ALBERS_SQKM',
    'ERP_P_202021', # Total population
    'ERP_P_202021 (%)', # Total population percentage
    'ERP_212021', # Residents per sqrt km
]

# Filter the table
filtered_population_data = population_data[columns].copy()

# Rename columns with mapping
mapping = {
    "SA2_NAME_2021": "SA2_name",
    "AREA_ALBERS_SQKM": "SA2_area",
    "ERP_P_202021": "SA2_population",
    "ERP_P_202021 (%)": "SA2_population_percentage",
    "ERP_212021": "SA2_population_density"
}

# Rename columns
SA2PopulationDistribution = filtered_population_data.rename(columns=mapping)

# Drop duplicate columns
SA2PopulationDistribution = SA2PopulationDistribution.loc[:,~SA2PopulationDistribution.columns.duplicated()].copy()

SA2PopulationDistribution.head()

,SA2_5DIG16,SA2_name,SA2_area,SA2_population,SA2_population_percentage,SA2_population_density
0,11007,Braidwood,3418.3525,4330.0,0.016856,1.3
1,11008,Karabar,6.9825,8546.0,0.033268,1223.9
2,11009,Queanbeyan,4.7620,11370.0,0.044262,2387.7
3,11010,Queanbeyan - East,13.0032,5093.0,0.019826,391.7
4,11012,Queanbeyan West - Jerrabomberra,13.6748,12743.0,0.049607,931.9


In [27]:
# Save the data as a csv file
file_path = os.path.join(savedir, "SA2PopulationData.csv")

SA2PopulationDistribution.to_csv(file_path, index=False)

### 3.2 Distances Hospitals - SA2
The next dataset is the hospital distances to SA2 areas. The dataset is created by adding the closest hospital ID to each SA2 area in the `data_distance_shortest` dataset. 

In [28]:
# Take a look at the data
data_distance_all.head()

,SA2_5DIG16,time_to_1,time_to_2,time_to_3,time_to_4,time_to_5,time_to_6,time_to_7,time_to_8,time_to_9,...,time_to_1002,time_to_1003,time_to_1004,time_to_1005,time_to_1006,time_to_1007,time_to_1008,time_to_1009,time_to_1010,time_to_1011
0,11007,158980.478696,55545.678696,55606.278696,55190.178696,55395.378696,11772.694738,159285.978696,32900.469892,18879.961833,...,18921.478696,176771.378696,16621.333137,32815.309343,30294.871355,23233.794482,7661.486679,29297.294482,155286.478696,12762.078696
1,11008,155655.689234,52220.889234,52281.489234,51865.389234,52070.589234,12091.796855,155961.189234,29592.889234,15555.389234,...,15596.689234,173446.589234,16932.096855,29498.889234,27762.141889,19910.789234,4287.389234,25974.289234,151961.689234,9437.289234
2,11009,155495.195328,52060.395328,52120.995328,51704.895328,51910.095328,11943.307067,155800.695328,29432.395328,15394.895328,...,15436.195328,173286.095328,16783.607067,29338.395328,27918.731659,19750.295328,4126.895328,25813.795328,151801.195328,9276.795328
3,11010,155499.714501,52064.914501,52125.514501,51709.414501,51914.614501,11855.214501,155805.214501,29436.914501,15399.414501,...,15440.714501,173290.614501,16695.514501,29342.914501,28018.014647,19754.814501,4131.414501,25818.314501,151805.714501,9281.314501
4,11011,156375.773471,52940.973471,53001.573471,52585.473471,52790.673471,12260.254294,156681.273471,30312.688050,16275.473471,...,16316.773471,174166.673471,17100.554294,30218.973471,28761.886848,20630.873471,5017.888440,26694.373471,152681.773471,10157.373471


In [29]:
# Take a look at the data
data_distance_shortest.head()

,SA2_5DIG16,shortest_time_sec,shortest_time_min
0,11007,1143.589117,20.0
1,11008,374.559538,7.0
2,11009,202.835640,4.0
3,11010,272.384746,5.0
4,11011,1693.438621,29.0


In [30]:
# Initialize a list to store the results
hospital_IDs = []
hospital_distances = []

# Iterate over the values list
for index, row in tqdm(data_distance_all.iterrows()):
    # Remove SA2_5DIG16 from the row
    filtered_row = row.drop("SA2_5DIG16")
    
    # Remove NaN values
    filtered_row = filtered_row.dropna()
    
    if len(filtered_row) == 0:
        hospital_IDs.append("Not found")
        hospital_distances.append("Not found")
        continue
    
    closests_hospitals = []
    closests_distances = []
    for i in range(0, 5):
        # Find the lowest value in the row
        results = filtered_row.idxmin()
        closests_hospitals.append(results.split("_")[2])
        closests_distances.append(str(filtered_row[results]))
        # Drop the lowest value
        filtered_row = filtered_row.drop(results)
        if len(filtered_row) == 0:
            break
        
    hospital_IDs.append(";".join(closests_hospitals))
    hospital_distances.append(";".join(closests_distances))

data_distance_shortest["closest_hospital_IDs"] = hospital_IDs
data_distance_shortest["closest_hospital_distances"] = hospital_distances

# Drop shortest_time_minutes column
data_distance_shortest = data_distance_shortest.drop(columns=["shortest_time_min"])

# Rename SA2_5DIG16 to SA2_5DIG
data_distance_shortest = data_distance_shortest.rename(columns={"SA2_5DIG16": "SA2_5DIG"})

2310it [00:02, 808.78it/s]


In [31]:
# Replace NaN values with "Not found"
data_distance_shortest = data_distance_shortest.fillna("Not found")

# Save the data as csv
file_path = os.path.join(savedir, "HospitalDistance.csv")
data_distance_shortest.to_csv(file_path, index=False)

### 3.3 Hospitals Metadata
The final dataset is the hospitals metadata. The dataset is created by renaming the columns of the `hospital_details` dataset.

In [32]:
# Inspect the data
hospital_details.head()

,Hospital name,Phone number,Street address,Suburb,Postcode,State,Local Hospital Network (LHN),Primary Health Network area (PHN),Website,Description,Sector,Beds,Latitude,Longitude,Hospital_ID
0,Abbotsford Private Hospital,08 9200 6282,61 Cambridge Street,West Leederville,6007,WA,NaN,NaN,www.abbotsfordhospital.com.au,NaN,Private,<50,-31.940992,115.837344,1
1,Adelaide Clinic,08 8269 8100,33 Park Terrace,Gilberton,5081,SA,NaN,NaN,www.adelaideclinic.com.au/,NaN,Private,50-99,-34.906580,138.613167,2
2,Adelaide Day Surgery Pty Ltd,08 8239 4900,18 North Terrace,Adelaide,5000,SA,NaN,NaN,http://www.curagroup.com.au/adelaide-day-surgery,NaN,Private,<50,-34.922409,138.588817,3
3,Adelaide Eye & Laser Centre,08 8274 7000,215 Greenhill Road,Eastwood,5063,SA,NaN,NaN,www.aelc.com.au,NaN,Private,NaN,-34.940420,138.621500,4
4,Adelaide Surgicentre,08 8211 0000,89 King William Street,Kent Town,5067,SA,NaN,NaN,www.asec.net.au,NaN,Private,<50,-34.917691,138.621022,5


In [33]:
# Create mapping
mapping = {
    "Hospital_ID": "hospital_ID",
    "Hospital name": "hospital_name",
    "Phone number": "phone_number",
    "Street address": "address",
    "Suburb": "suburb",
    "Postcode": "postcode",
    "State": "state",
    "Local Hospital Network (LHN)": "local_hospital_network",
    "Primary Health Network area (PHN)": "primary_health_network",
    "Website": "website",
    "Description": "description",
    "Sector": "sector",
    "Beds": "beds",
    "Latitude": "latitude",
    "Longitude": "longitude"
}

# Apply mapping
HospitalMetadata = hospital_details.rename(columns=mapping)
HospitalMetadata.head()

,hospital_name,phone_number,address,suburb,postcode,state,local_hospital_network,primary_health_network,website,description,sector,beds,latitude,longitude,hospital_ID
0,Abbotsford Private Hospital,08 9200 6282,61 Cambridge Street,West Leederville,6007,WA,NaN,NaN,www.abbotsfordhospital.com.au,NaN,Private,<50,-31.940992,115.837344,1
1,Adelaide Clinic,08 8269 8100,33 Park Terrace,Gilberton,5081,SA,NaN,NaN,www.adelaideclinic.com.au/,NaN,Private,50-99,-34.906580,138.613167,2
2,Adelaide Day Surgery Pty Ltd,08 8239 4900,18 North Terrace,Adelaide,5000,SA,NaN,NaN,http://www.curagroup.com.au/adelaide-day-surgery,NaN,Private,<50,-34.922409,138.588817,3
3,Adelaide Eye & Laser Centre,08 8274 7000,215 Greenhill Road,Eastwood,5063,SA,NaN,NaN,www.aelc.com.au,NaN,Private,NaN,-34.940420,138.621500,4
4,Adelaide Surgicentre,08 8211 0000,89 King William Street,Kent Town,5067,SA,NaN,NaN,www.asec.net.au,NaN,Private,<50,-34.917691,138.621022,5


In [34]:
# Save the data as csv
file_path = os.path.join(savedir, "HospitalMetadata.csv")
HospitalMetadata.to_csv(file_path, index=False)

## 4 Knowledge Graph
### 4.1 Libraries and Functions

In [42]:
from lib.GraphDB import GraphDB

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "healthcare"  # Change this to your Neo4j database password
DBNAME = "maingraph"
  
graph_handler = GraphDB(uri=URI, user=USER, password=PASSWORD, database=DBNAME)

### 4.2 Creating the Knowledge Graph
#### 4.2.1 Creating the Nodes
State areas (level 2)

In [43]:
file_path = os.path.join(savedir, "SA2PopulationData.csv")
SA2PopulationDistribution = pd.read_csv(file_path)

# Use the GraphDB class to create nodes for all rows in the dataframe
for index, row in tqdm(SA2PopulationDistribution.iterrows()):
    graph_handler.create_sa2(
        sa2_5dig=row["SA2_5DIG16"], 
        sa2_name=row["SA2_name"], 
        area=float(row["SA2_area"]),
        population=float(row["SA2_population"]),
        pop_percentage=float(row["SA2_population_percentage"]),
        pop_density=float(row["SA2_population_density"])
        )

2473it [00:06, 400.20it/s]


Hospitals

In [44]:
file_path = os.path.join(savedir, "HospitalMetadata.csv")
HospitalMetadata = pd.read_csv(file_path)

for index, row in tqdm(HospitalMetadata.iterrows()):
    graph_handler.create_hospital(
        hospital_id=str(row["hospital_ID"]),
        hospital_name=row["hospital_name"],
        phone_number=row["phone_number"],
        address=row["address"],
        suburb=row["suburb"],
        postcode=row["postcode"],
        state=row["state"],
        lhn=row["local_hospital_network"],
        phn=row["primary_health_network"],
        website=row["website"],
        description=row["description"],
        sector=row["sector"],
        beds=row["beds"],
        latitude=row["latitude"],
        longitude=row["longitude"])

1011it [00:03, 294.38it/s]


#### 4.2.2 Creating the Edges
Hospital - SA2

In [45]:
file_path = os.path.join(savedir, "HospitalDistance.csv")
HospitalDistance = pd.read_csv(file_path)

file_path = os.path.join(savedir, "AccessibilityEdges.csv")
AccessibilityEdges = pd.read_csv(file_path)

for index, row in tqdm(HospitalDistance.iterrows()):
    sa2_5dig=str(row["SA2_5DIG"])
    hospital_ids=row["closest_hospital_IDs"].split(";")
    hospital_distances=row["closest_hospital_distances"].split(";")
    for i, hospital_id in enumerate(hospital_ids):
        if hospital_id == "Not found":
            continue
        row = AccessibilityEdges[
            (AccessibilityEdges["SA2_5DIG"] == int(sa2_5dig)) &
            (AccessibilityEdges["hospital_ID"] == int(hospital_id))
        ]
        if len(row) == 0:
            continue
        accessible = row["accessible"].values[0]
        further_than_2h = row["further_than_2h"].values[0]
        graph_handler.add_relation_sa2_hospital(
            sa2_5dig=sa2_5dig,
            hospital_id=str(hospital_id),
            distance_time=hospital_distances[i],
            accessible=accessible,
            further_than_2h=further_than_2h
        )
    

2310it [02:59, 12.89it/s]


#### 4.2.3 Cleaning up

In [46]:
graph_handler.close()